In [ ]:
export PYTHONPATH=$PYTHONPATH:'/home/

In [25]:
# -*- coding: utf-8 -*-
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Convert raw PASCAL dataset to TFRecord for object_detection.

Example usage:
    python object_detection/dataset_tools/create_pascal_tf_record.py \
        --data_dir=/home/user/VOCdevkit \
        --year=VOC2012 \
        --output_path=/home/user/pascal.record
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import io
import logging
import os
import sys

sys.path.append('/home/pmp/myoungjin/sp-deep-learning/models/research')
sys.path.append('/home/pmp/myoungjin/sp-deep-learning/models/research/slim')

import random


from lxml import etree
import PIL.Image
import tensorflow as tf

#import dataset_util
#import label_map_util

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

'''
flags = tf.app.flags
flags.DEFINE_string('data_dir', '', 'Root directory to raw PASCAL VOC dataset.')

flags.DEFINE_string('annotations_dir', 'Annotations',
                    '(Relative) path to annotations directory.')

flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('label_map_path', 'data/pascal_label_map.pbtxt',
                    'Path to label map proto')
flags.DEFINE_boolean('ignore_difficult_instances', False, 'Whether to ignore '
                     'difficult instances')


FLAGS = flags.FLAGS
'''

def dict_to_tf_example(data,
                       dataset_directory,
                       label_map_dict,ignore_difficult_instances):
                       #ignore_difficult_instances=False) :
                       #image_subdirectory='hangul-images'):
  """Convert XML derived dict to tf.Example proto.

  Notice that this function normalizes the bounding box coordinates provided
  by the raw data.

  Args:
    data: dict holding PASCAL XML fields for a single image (obtained by
      running dataset_util.recursive_parse_xml_to_dict)
    dataset_directory: Path to root directory holding PASCAL dataset
    label_map_dict: A map from string label names to integers ids.
    ignore_difficult_instances: Whether to skip difficult instances in the
      dataset  (default: False).
    image_subdirectory: String specifying subdirectory within the
      PASCAL dataset directory holding the actual image data.

  Returns:
    example: The converted tf.Example.

  Raises:
    ValueError: if the image pointed to by data['filename'] is not a valid JPEG
  """
  #img_path = os.path.join(data['folder'], image_subdirectory, data['filename'])
  #full_path = os.path.join(dataset_directory, img_path)
  full_path = dataset_directory + data['folder']+'/'+ data['filename']
  with tf.gfile.GFile(full_path, 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = PIL.Image.open(encoded_jpg_io)
  if image.format != 'JPEG':
    raise ValueError('Image format not JPEG')
  key = hashlib.sha256(encoded_jpg).hexdigest()
  
  width = int(data['size']['width'])
  height = int(data['size']['height'])

  xmin = []
  ymin = []
  xmax = []
  ymax = []
  classes = []
  classes_text = []
  truncated = []
  poses = []
  difficult_obj = []
  
  if 'object' in data:
    for obj in data['object']:
      difficult = bool(int(obj['difficult']))
      if ignore_difficult_instances and difficult:
        continue

      difficult_obj.append(int(difficult))

      xmin.append(float(obj['bndbox']['xmin']) / width)
      ymin.append(float(obj['bndbox']['ymin']) / height)
      xmax.append(float(obj['bndbox']['xmax']) / width)
      ymax.append(float(obj['bndbox']['ymax']) / height)
      classes_text.append(obj['name'].encode('utf8'))
      classes.append(label_map_dict[obj['name']])
      truncated.append(int(obj['truncated']))
      poses.append(obj['pose'].encode('utf8'))
      '''
      print(data)
      print(xmin)
      print(ymin)
      print(xmax)
      print(ymax)
      print(classes_text)
      print(classes)
      '''
  example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(
          data['filename'].encode('utf8')),
      'image/source_id': dataset_util.bytes_feature(
          data['filename'].encode('utf8')),
      'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
      'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
      'image/object/truncated': dataset_util.int64_list_feature(truncated),
      'image/object/view': dataset_util.bytes_list_feature(poses),
  }))
  return example


def main(_):
  

  #data_dir = FLAGS.data_dir
  
  data_dir = '/home/pmp/myoungjin/sp-deep-learning/dataset/data/image-data/'
  output_path = '/home/pmp/myoungjin/sp-deep-learning/dataset/data/tfdata/hangul.record'
  data_sub = 'image-data'

  writer = tf.python_io.TFRecordWriter(output_path)
  #label_map_path = data_dir+'/hangul1.pbtxt'
  #label_map_dict = label_map_util.get_label_map_dict(label_map_path)
  label_map_dict = {};
  f = open('/home/pmp/myoungjin/sp-deep-learning/dataset/labels/2350-common-hangul.txt','r')
  idx = 1
  while True:
    line = f.readline()
    if not line: break
    if idx == 1:
      ch = '가'    
    else :  
      ch = line[0]    
    label_map_dict[ch] = idx
    idx = idx +1
  f.close()
  
  print(label_map_dict)  
  logging.info('Reading from hangul dataset.')
  #annotat_dir = '/annotation/hangul-annotation'
  annotations_dir = '/home/pmp/myoungjin/sp-deep-learning/dataset/data/annotation/hangul-annotation'
    
  examples = os.listdir(annotations_dir)
    
  random.shuffle(examples)
   
  index = 0  
  for example in examples:
    index = index +1   
    if index % 100 == 0:
      logging.info('On image %d of %d', idx, len(examples))
    #path = os.path.join(annotations_dir, example + '.xml')
    path = os.path.join(annotations_dir,example)
    with tf.gfile.GFile(path, 'r') as fid:
      xml_str = fid.read()
    xml = etree.fromstring(xml_str)
    data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']

    tf_example = dict_to_tf_example(data, data_dir, label_map_dict,False)
    #dict_to_tf_example(data, data_dir, label_map_dict,False)#,FlAGS.ignore_difficult_instances)
    writer.write(tf_example.SerializeToString())

  writer.close()
  

if __name__ == '__main__':
  tf.app.run()



{'가': 1, '각': 2, '간': 3, '갇': 4, '갈': 5, '갉': 6, '갊': 7, '감': 8, '갑': 9, '값': 10, '갓': 11, '갔': 12, '강': 13, '갖': 14, '갗': 15, '같': 16, '갚': 17, '갛': 18, '개': 19, '객': 20, '갠': 21, '갤': 22, '갬': 23, '갭': 24, '갯': 25, '갰': 26, '갱': 27, '갸': 28, '갹': 29, '갼': 30, '걀': 31, '걋': 32, '걍': 33, '걔': 34, '걘': 35, '걜': 36, '거': 37, '걱': 38, '건': 39, '걷': 40, '걸': 41, '걺': 42, '검': 43, '겁': 44, '것': 45, '겄': 46, '겅': 47, '겆': 48, '겉': 49, '겊': 50, '겋': 51, '게': 52, '겐': 53, '겔': 54, '겜': 55, '겝': 56, '겟': 57, '겠': 58, '겡': 59, '겨': 60, '격': 61, '겪': 62, '견': 63, '겯': 64, '결': 65, '겸': 66, '겹': 67, '겻': 68, '겼': 69, '경': 70, '곁': 71, '계': 72, '곈': 73, '곌': 74, '곕': 75, '곗': 76, '고': 77, '곡': 78, '곤': 79, '곧': 80, '골': 81, '곪': 82, '곬': 83, '곯': 84, '곰': 85, '곱': 86, '곳': 87, '공': 88, '곶': 89, '과': 90, '곽': 91, '관': 92, '괄': 93, '괆': 94, '괌': 95, '괍': 96, '괏': 97, '광': 98, '괘': 99, '괜': 100, '괠': 101, '괩': 102, '괬': 103, '괭': 104, '괴': 105, '괵': 106, '괸': 107, '괼': 108, '굄': 109, '굅': 110, '굇': 11

KeyboardInterrupt: 

In [21]:
u = '고'
b3 = u.encode("utf-8")
print(b3)

b'\xea\xb3\xa0'
